# Opptaksområder 2019

In [250]:
aargang <- 2020

In [251]:
options(repr.matrix.max.rows=600, repr.matrix.max.cols=2000)

suppressPackageStartupMessages({ 
library(tidyverse)
library(readxl)
library(klassR)
library(sf)
library(leaflet)
        })

## Kodeliste for opptaksområder i spesialisthelsetjenesten (somatikk)

In [252]:
opptaksomrader_KLASS <- klassR::GetKlass(629, output_style = "wide", date = c(paste0(aargang, "-01-01"))) %>%
dplyr::rename(GRUNNKRETSNUMMER = code4, 
             GRUNNKRETS_NAVN = name4, 
             OPPTAK_NUMMER = code3, 
             OPPTAK = name3, 
             ORGNR_HF = code2, 
             NAVN_HF = name2, 
             ORGNR_RHF = code1, 
             NAVN_RHF = name1)

nrow(opptaksomrader_KLASS)

[1] 14457

In [253]:
opptaksomrader_KLASS %>%
dplyr::filter(GRUNNKRETSNUMMER == "30039999")

GRUNNKRETSNUMMER,GRUNNKRETS_NAVN,OPPTAK_NUMMER,OPPTAK,ORGNR_HF,NAVN_HF,ORGNR_RHF,NAVN_RHF
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
30039999,Uoppgitt grunnkrets,48,Østfold,983971768,SYKEHUSET ØSTFOLD HF,991324968,HELSE SØR-ØST RHF


### Korrespondanse mellom t og t-1 fra KLASS

In [254]:
grunnkrets_KLASS <- klassR::GetKlass(1, date = c(paste0(aargang-1, "-01-01"), paste0(aargang, "-01-01")), correspond = TRUE) %>%
dplyr::rename(GRUNNKRETSNUMMER_T1 = sourceCode, 
             GRUNNKRETSNUMMER = targetCode, 
             targetName = targetName)

nrow(grunnkrets_KLASS)

grunnkrets_KLASS <- grunnkrets_KLASS %>%
dplyr::filter(!GRUNNKRETSNUMMER_T1 %in% c("18500111") |  !GRUNNKRETSNUMMER %in% c("18061610"), # Fjerner Kjerrvika fra Storå
             !GRUNNKRETSNUMMER_T1 %in% c("18500109") |  !GRUNNKRETSNUMMER %in% c("18750211"),  # Fjerner Indre Tysfjord fra Kjerrvika
             !GRUNNKRETSNUMMER_T1 %in% c("18500119") |  !GRUNNKRETSNUMMER %in% c("18750211")) # Fjerner Bjørntoppen fra Kjerrvika

nrow(grunnkrets_KLASS)

[1] 10120

[1] 10117

### Merger opptaksområder med KLASS (t-1)

In [269]:
opptaksomrader_KLASS_2 <- opptaksomrader_KLASS %>%
dplyr::left_join(grunnkrets_KLASS, by = "GRUNNKRETSNUMMER") %>%
dplyr::mutate(GRUNNKRETSNUMMER_T1 = case_when(
is.na(GRUNNKRETSNUMMER_T1) ~ GRUNNKRETSNUMMER, 
    TRUE ~ GRUNNKRETSNUMMER_T1
)) %>%
dplyr::select(GRUNNKRETSNUMMER, GRUNNKRETSNUMMER_T1, GRUNNKRETS_NAVN, OPPTAK_NUMMER, OPPTAK, ORGNR_HF, NAVN_HF, ORGNR_RHF, NAVN_RHF) %>%
dplyr::distinct(GRUNNKRETSNUMMER_T1, OPPTAK_NUMMER, OPPTAK, ORGNR_HF, NAVN_HF, ORGNR_RHF, NAVN_RHF)

nrow(opptaksomrader_KLASS_2)

[1] 14424

In [270]:
# opptaksomrader_KLASS_2 %>%
# dplyr::filter(GRUNNKRETSNUMMER_T1 == "18500109")

### Laster inn KLASS (t-1)

In [271]:
grunnkrets_KLASS_T1 <- klassR::GetKlass(1, output_style = "wide", date = c(paste0(aargang-1, "-01-01"))) %>%
dplyr::rename(GRUNNKRETSNUMMER_T1 = code2, 
             GRUNNKRETS_NAVN_T1 = name2) %>%
dplyr::select(GRUNNKRETSNUMMER_T1, GRUNNKRETS_NAVN_T1) 

nrow(grunnkrets_KLASS_T1)
nrow(opptaksomrader_KLASS_2)

[1] 14489

[1] 14424

In [272]:
# opptaksomrader_KLASS_2 %>%
# group_by(GRUNNKRETSNUMMER_T1) %>% # grunnkrets_KLASS_T1_2
# filter(n()>1)

In [274]:
grunnkrets_KLASS_T1_2 <- grunnkrets_KLASS_T1 %>%
dplyr::left_join(opptaksomrader_KLASS_2, by = "GRUNNKRETSNUMMER_T1") %>%
dplyr::filter(!is.na(OPPTAK))

nrow(grunnkrets_KLASS_T1_2)

head(grunnkrets_KLASS_T1_2)

# grunnkrets_KLASS_T1_2 <- grunnkrets_KLASS_T1 %>%
# dplyr::distinct(GRUNNKRETSNUMMER_T1, GRUNNKRETS_NAVN_T1, OPPTAK)

[1] 14180

,GRUNNKRETSNUMMER_T1,GRUNNKRETS_NAVN_T1,OPPTAK_NUMMER,OPPTAK,ORGNR_HF,NAVN_HF,ORGNR_RHF,NAVN_RHF
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,01010102,Båstadlund,48,Østfold,983971768,SYKEHUSET ØSTFOLD HF,991324968,HELSE SØR-ØST RHF
2,01010103,Låby,48,Østfold,983971768,SYKEHUSET ØSTFOLD HF,991324968,HELSE SØR-ØST RHF
3,01010104,Stangeløkka,48,Østfold,983971768,SYKEHUSET ØSTFOLD HF,991324968,HELSE SØR-ØST RHF
4,01010105,Refne/Banken,48,Østfold,983971768,SYKEHUSET ØSTFOLD HF,991324968,HELSE SØR-ØST RHF
5,01010107,Remmen,48,Østfold,983971768,SYKEHUSET ØSTFOLD HF,991324968,HELSE SØR-ØST RHF
6,01010108,Refne,48,Østfold,983971768,SYKEHUSET ØSTFOLD HF,991324968,HELSE SØR-ØST RHF


### Lagrer filen 

In [301]:
test <- grunnkrets_KLASS_T1_2 %>%
  dplyr::mutate(TOM_FORELDER = "") %>%
  # dplyr::filter(opptak %in% c("Stavanger", "Ålesund")) %>%
  dplyr::select(GRUNNKRETSNUMMER_T1, GRUNNKRETS_NAVN_T1, OPPTAK_NUMMER, ORGNR_HF, NAVN_HF, OPPTAK, ORGNR_RHF, NAVN_RHF, TOM_FORELDER)

level_1 <- test %>%
  dplyr::select(ORGNR_RHF, TOM_FORELDER, NAVN_RHF) %>%
  dplyr::rename('ns1:kode' = ORGNR_RHF, 
                'ns1:forelder' = TOM_FORELDER, 
                'ns1:navn_bokmål' = NAVN_RHF) %>%
  dplyr::distinct()

level_2 <- test %>%
  dplyr::select(ORGNR_HF, ORGNR_RHF, NAVN_HF) %>%
  dplyr::rename('ns1:kode' = ORGNR_HF, 
                'ns1:forelder' = ORGNR_RHF, 
                'ns1:navn_bokmål' = NAVN_HF) %>%
  dplyr::distinct()

level_3 <- test %>%
  dplyr::select(OPPTAK_NUMMER, ORGNR_HF, OPPTAK) %>%
  dplyr::rename('ns1:kode' = OPPTAK_NUMMER, 
                'ns1:forelder' = ORGNR_HF, 
                'ns1:navn_bokmål' = OPPTAK) %>%
  dplyr::distinct()

level_4 <- test %>%
  dplyr::select(GRUNNKRETSNUMMER_T1, OPPTAK_NUMMER, GRUNNKRETS_NAVN_T1) %>%
  dplyr::rename('ns1:kode' = GRUNNKRETSNUMMER_T1, 
                'ns1:forelder' = OPPTAK_NUMMER, 
                'ns1:navn_bokmål' = GRUNNKRETS_NAVN_T1) %>%
  dplyr::distinct()

KLASS <- rbind(level_1, level_2, level_3, level_4)

In [302]:
openxlsx::write.xlsx(KLASS, file = "/ssb/bruker/rdn/opptak_2019.xlsx",
                     rowNames = FALSE,
                     showNA = FALSE,
                     overwrite=T) # T = overskriver dersom filen allerede finnes, F = gir feilmelding dersom filen finnes

In [290]:
uoppgitt_grunnkrets <- grunnkrets_KLASS_T1 %>%
dplyr::left_join(opptaksomrader_KLASS_2, by = "GRUNNKRETSNUMMER_T1") %>%
dplyr::filter(is.na(OPPTAK)) %>%
dplyr::select(GRUNNKRETSNUMMER_T1)

nrow(uoppgitt_grunnkrets)

head(uoppgitt_grunnkrets)

kommune_test <- grunnkrets_KLASS_T1_2 %>%
# dplyr::mutate(KOMMUNENUMMER = substr(GRUNNKRETSNUMMER_T1, 1, 4)) %>%
# dplyr::group_by(KOMMUNENUMMER, OPPTAK_NUMMER, OPPTAK, ORGNR_HF, NAVN_HF, ORGNR_RHF, NAVN_RHF) %>% # OPPTAK_NUMMER, OPPTAK, ORGNR_HF, NAVN_HF, ORGNR_RHF, NAVN_RHF
# dplyr::tally() %>%
# arrange(KOMMUNENUMMER) %>%
# dplyr::group_by(KOMMUNENUMMER, OPPTAK_NUMMER, OPPTAK, ORGNR_HF, NAVN_HF, ORGNR_RHF, NAVN_RHF) %>% # 
# slice(which.max(n)) %>%
# dplyr::mutate(GRUNNKRETSNUMMER_T1 = paste0(KOMMUNENUMMER, "9999")) %>%
# ungroup() %>%
# dplyr::select(-KOMMUNENUMMER) %>%
dplyr::filter(GRUNNKRETSNUMMER_T1 %in% unique(uoppgitt_grunnkrets$GRUNNKRETSNUMMER_T1))

nrow(kommune_test)

# head(kommune_test)

[1] 309

,GRUNNKRETSNUMMER_T1
,<chr>
1,01019999
2,01049999
3,01059999
4,01069999
5,01119999
6,01189999


[1] 0

In [248]:
# kommune_test <- grunnkrets_KLASS_T1 %>%
# dplyr::filter(!is.na(OPPTAK)) %>%
# dplyr::mutate(KOMMUNENUMMER = substr(GRUNNKRETSNUMMER_T1, 1, 4)) %>%
# dplyr::group_by(KOMMUNENUMMER, OPPTAK_NUMMER, OPPTAK, ORGNR_HF, NAVN_HF, ORGNR_RHF, NAVN_RHF) %>%
# dplyr::tally() %>%
# arrange(KOMMUNENUMMER) %>%
# slice(which.max(n)) %>%
# dplyr::mutate(GRUNNKRETSNUMMER_T1 = paste0(KOMMUNENUMMER, "9999")) %>%
# ungroup() %>%
# dplyr::select(-KOMMUNENUMMER, -n)

# nrow(kommune_test)

# uoppgitt_grunnkrets <- grunnkrets_KLASS_T1 %>%
# dplyr::filter(is.na(OPPTAK)) %>%
# dplyr::select(GRUNNKRETSNUMMER_T1)

# nrow(uoppgitt_grunnkrets)

# uoppgitt_grunnkrets %>%
# dplyr::filter(GRUNNKRETSNUMMER_T1 == "02269999")

# uoppgitt_grunnkrets <- uoppgitt_grunnkrets %>%
# dplyr::left_join(kommune_test, by = "GRUNNKRETSNUMMER_T1")

# uoppgitt_grunnkrets %>%
# dplyr::filter(GRUNNKRETSNUMMER_T1 == "02269999")

# nrow(uoppgitt_grunnkrets)

In [258]:
# uoppgitt_grunnkrets %>%
# group_by(GRUNNKRETSNUMMER_T1) %>% # grunnkrets_KLASS_T1_2
# filter(n()>1)

In [259]:
# kommune_test %>%
# group_by(KOMMUNENUMMER) %>% # grunnkrets_KLASS_T1_2
# filter(n()>1)

In [260]:
# grunnkrets_KLASS_T1_2 %>%
# # dplyr::distinct(GRUNNKRETSNUMMER_T1, GRUNNKRETS_NAVN_T1, OPPTAK) %>%
# group_by(GRUNNKRETSNUMMER_T1) %>% # grunnkrets_KLASS_T1_2
# filter(n()>1)

In [ ]:
# grunnkrets_KLASS_T1_2 <- grunnkrets_KLASS_T1 %>%
# dplyr::mutate(OPPTAK = case_when(
# GRUNNKRETS_NAVN_T1 == "Indre Tysfjord" ~ "Narvik",
# GRUNNKRETS_NAVN_T1 == "Kjerrvika" ~ "Narvik",
# GRUNNKRETS_NAVN_T1 == "Bjørntoppen" ~ "Narvik",
# TRUE ~ OPPTAK
# )) 

grunnkrets_KLASS_T1_2 %>%
dplyr::distinct(GRUNNKRETSNUMMER_T1, GRUNNKRETS_NAVN_T1, OPPTAK) %>%
group_by(GRUNNKRETSNUMMER_T1) %>% # grunnkrets_KLASS_T1_2
filter(n()>1)

nrow(grunnkrets_KLASS_T1_2)

In [155]:
# grunnkrets_KLASS_T1_2 %>%
# dplyr::filter(substr(GRUNNKRETSNUMMER_T1, 1, 4) == "0226") # Akershus

In [157]:
# # grunnkrets_KLASS_T1_2 %>%
# # # dplyr::filter(GRUNNKRETSNUMMER_T1 == "01059999")
# # dplyr::filter(GRUNNKRETSNUMMER == "30039999")

# grunnkrets_KLASS_T1_2 %>%
# dplyr::filter(!is.na(OPPTAK)) %>%
# dplyr::mutate(KOMMUNENUMMER = substr(GRUNNKRETSNUMMER_T1, 1, 4)) %>%
# dplyr::select(OPPTAK, KOMMUNENUMMER) %>%
# dplyr::distinct() %>%
# group_by(KOMMUNENUMMER) %>% # grunnkrets_KLASS_T1_2
# filter(n()>1)

OBS: Uoppgitt grunnkrets per kommune?

In [158]:
grunnkrets_KLASS_T1_2 %>%
dplyr::filter(is.na(GRUNNKRETSNUMMER)) %>%
dplyr::filter(substr(GRUNNKRETSNUMMER_T1, 5, 8) != "9999")

GRUNNKRETSNUMMER_T1,GRUNNKRETS_NAVN_T1,GRUNNKRETSNUMMER,OPPTAK,ORGNR_HF,NAVN_HF,ORGNR_RHF,NAVN_RHF
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>


In [159]:
# nrow(opptaksomrader_KLASS_2)

nrow(opptaksomrader_KLASS_2)

opptaksomrader_KLASS_2 %>%
dplyr::filter(GRUNNKRETSNUMMER_T1 == "18500109")

# opptaksomrader_KLASS_2 %>%
# dplyr::filter(is.na(OPPTAK_NUMMER))

# opptaksomrader_KLASS_2 %>%
# dplyr::filter(is.na(GRUNNKRETSNUMMER_T1))

[1] 14458

GRUNNKRETSNUMMER,GRUNNKRETSNUMMER_T1,OPPTAK,ORGNR_HF,NAVN_HF,ORGNR_RHF,NAVN_RHF
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
18061609,18500109,Narvik,983974899,UNIVERSITETSSYKEHUSET NORD-NORGE HF,883658752,HELSE NORD RHF


In [160]:
test <- opptaksomrader_KLASS_2 %>%
group_by(GRUNNKRETSNUMMER_T1) %>% 
filter(n()>1)

nrow(test)

[1] 57

In [161]:
test

GRUNNKRETSNUMMER,GRUNNKRETSNUMMER_T1,OPPTAK,ORGNR_HF,NAVN_HF,ORGNR_RHF,NAVN_RHF
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
42040312,10010311,Kristiansand,983975240,SØRLANDET SYKEHUS HF,991324968,HELSE SØR-ØST RHF
42040313,10010311,Kristiansand,983975240,SØRLANDET SYKEHUS HF,991324968,HELSE SØR-ØST RHF
42040509,10010507,Kristiansand,983975240,SØRLANDET SYKEHUS HF,991324968,HELSE SØR-ØST RHF
42040510,10010507,Kristiansand,983975240,SØRLANDET SYKEHUS HF,991324968,HELSE SØR-ØST RHF
42040806,10010805,Kristiansand,983975240,SØRLANDET SYKEHUS HF,991324968,HELSE SØR-ØST RHF
42040807,10010805,Kristiansand,983975240,SØRLANDET SYKEHUS HF,991324968,HELSE SØR-ØST RHF
42040918,10010914,Kristiansand,983975240,SØRLANDET SYKEHUS HF,991324968,HELSE SØR-ØST RHF
42040919,10010914,Kristiansand,983975240,SØRLANDET SYKEHUS HF,991324968,HELSE SØR-ØST RHF
42040920,10010906,Kristiansand,983975240,SØRLANDET SYKEHUS HF,991324968,HELSE SØR-ØST RHF
